In [46]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
def coletar_links(url, limite):
    # Fila contém uma série de links (todos os links das páginas acessadas). Set contém os links únicos já visitados.
    fila = [url]
    visitados = set()
    salvos = 0

    # Se menos de 5k links foram guardados, conitnua execução
    while salvos < limite:
        try:
            # Acessa o último link inserido na fila
            atual = fila.pop()

            # Verifica se esse link já foi acessado. Se não, continua a execução e adiciona a lista de visitados.
            if atual in visitados: 
                continue
            visitados.add(atual)

            # Carrega a página atual.
            req = requests.get(atual)
            soup = BeautifulSoup(req.text, 'html.parser')

            titulo = soup.select(".mw-page-title-main")
            nome_arquivo = titulo[0].text

            with open(f"pag/{nome_arquivo}.html", "w", encoding="utf-8") as file:
                conteudo = req.content.decode("utf-8")
                file.write(conteudo)
            salvos += 1

            # Itera sobre todos os links da página atual.
            for link in soup.find_all('a', href=True):

                # Verifica se a página é um verbete.
                pagina = link['href']
                if pagina.startswith('/wiki/') and ':' not in pagina:

                    # Se algum dos links da página carregada já tiver sido visitada, pula.
                    link_atual = 'https://pt.wikipedia.org' + pagina
                    if link_atual in visitados:
                        continue
                    # Adiciona todos que forem verbetes à fila. O último link coletado será a próxima página a ser aberta.
                    fila.append('https://pt.wikipedia.org' + pagina)

            print(salvos, end="\r")
            time.sleep(0.6)

        # Evita páginas sem título (titulo[0].text)
        except IndexError:
            continue

        # Evita erros de mal formatação de título
        except FileNotFoundError:
            continue

    return list(visitados)

links = coletar_links('https://pt.wikipedia.org', 5000)


## Tarefa 1

In [17]:
import os
import csv

print(len(os.listdir("pages/")))
with open("links.csv", "w", newline="") as file:
    writer = csv.writer(file)
    fields = ["links"]
    writer.writerow(fields)
    for link in links:
        writer.writerow([link])

## Tarefa 2

In [ ]:
import json
def create_json(url):

    response = requests.get(url)
    bs = BeautifulSoup(response.content, "html.parser")

    conteudo_principal = bs.find("div", class_="mw-parser-output")
    tabela = conteudo_principal.find("table", class_="infobox infobox_v2") if conteudo_principal else None

    if(tabela):

        trs = tabela.find_all("tr")
        titulo = trs[0].text.strip()
        titulo = re.sub(r'[^\w\s-]', '', titulo)  # remove caracteres especiais
        titulo = re.sub(r'\s+', '_', titulo)      # substitui espaços por underline

        dicionario = {}
        for tr in trs:
            tds = tr.find_all('td')

            if(len(tds) == 2):
                div = tds[1].find("div")

                if(div):
                    ul = div.find("ul")
                    
                    if(ul):
                        #print(f'TEM LISTAAA!!!! -----> {tds[0].text.strip()}')
                        lis = ul.find_all('li')
                        data_lista = []
                        try: #caso haja paragrafo E TAMBEM lista para a mesma chave
                            data_lista.append(tds[1].span.text.strip())
                        except:
                            continue
                        for li in lis:
                            data_lista.append(li.text.strip())
                            #print(f'colocando {li.text.strip()} na lista {tds[0].text.strip()}')
                        dicionario[tds[0].text.strip()] = data_lista
                else:
                    dicionario[tds[0].text.strip()] = tds[1].text.strip()
        with open(f"jsons/{titulo}.json", 'w') as f:
            json.dump(dicionario, f, indent=2, ensure_ascii=False)

In [45]:
for link in links:
    create_json(link)